# **KASDD F1 Lap time - Biasa Aja**

# Import Dataset

In [ ]:
# import library dan data
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import scipy.stats as scp
import scipy.cluster.hierarchy as shc

from decimal import Decimal
from numpy.polynomial.polynomial import polyfit
from sklearn.preprocessing import LabelEncoder, Normalizer, StandardScaler, MinMaxScaler
from sklearn.feature_selection import chi2, mutual_info_regression, mutual_info_classif, SelectKBest, mutual_info_regression, SelectPercentile, mutual_info_regression, f_classif
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, KFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB, MultinomialNB, CategoricalNB
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold
from sklearn.cluster import KMeans
from yellowbrick.cluster import SilhouetteVisualizer, KElbowVisualizer
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import RepeatedStratifiedKFold


f1_data = pd.read_csv("f1_train.csv")
f1_data = f1_data.drop(axis=1, columns=["ID"])
f1_data

In [ ]:
f1_data.info()

# Preprocessing

In [ ]:
def cek_duplicates(df):
    if df.duplicated().sum() > 0:
        print("Terdapat", df.duplicated().sum(), "pasang data yang redundan")
        display(df[df.duplicated()])
    else:
        print("Tidak ada data yang redundan")

def cek_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na*100 / len(df)

    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")

    else:
        print(missing_data[missing_data['Total'] > 0])

def cek_outlier(df):
    df_numerical = df.select_dtypes(include=['float64', 'int64']) 
    Q1 = df_numerical.quantile(0.25, numeric_only=True)
    Q3 = df_numerical.quantile(0.75, numeric_only=True)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df_numerical < lower_limit) | (df_numerical > upper_limit)

    # Menghitung dan menampilkan persentase outlier pada tiap atribut.
    percentage_outliers = (outliers.sum() / len(df)) * 100
    print("Persentase Outlier pada tiap atribut:")
    print(percentage_outliers)

### Handling Missing Value

In [ ]:
cek_null(f1_data)

> Atribut `PitOutTime`, `PitInTime`, dan `DeletedReason` memiliki persentase jumlah missing value mencapai lebih dari 90%. Oleh karena itu, atribut-atribut tersebut perlu di drop.

In [ ]:
f1_data = f1_data.drop(axis=1, columns=['PitOutTime', 'PitInTime', 'DeletedReason'])

In [ ]:
def show_data_type_and_distribution(df):
    print("Menampilkan informasi dataset dan tipe data")
    print('#'*50)
    df.info()
    print('#'*50)
    print("Menampilkan distribusi data numerik")
    numerics = ['SpeedI2', 'SpeedFL', 'SpeedST', 'SpeedI1']
    for col in numerics:
        df_feature = f1_data[col]
        plt.figure(figsize=(10,5))
        plt.title(f'Distribusi data {col}')
        sns.kdeplot(df_feature, fill=True)
        plt.show()
    print("Menampilkan Modus dari data kategorikal")
    categoricals = ["IsPersonalBest", "Sector2SessionTime", "Sector2Time","Sector3SessionTime", "Sector3Time", "LapTime", "Sector1Time", "Sector1SessionTime"]
    for col in categoricals:
        print(f"Mode for {col} = {f1_data[col].mode()[0]}")

In [ ]:
show_data_type_and_distribution(f1_data)

In [ ]:
def fill_null_mean(df):
    mean = df.mean()
    filled_df = df.fillna(mean)
    return filled_df
def fill_null_median(df):
    median = df.median()
    filled_df = df.fillna(median)
    return filled_df
def fill_null_mode(df):
    mode = df.mode()[0]
    filled_df = df.fillna(mode)
    return filled_df

In [ ]:
f1_data['SpeedI2'] = fill_null_median(f1_data['SpeedI2'])
f1_data['SpeedFL'] = fill_null_median(f1_data['SpeedFL'])
f1_data['SpeedST'] = fill_null_mean(f1_data['SpeedST'])
f1_data['SpeedI1'] = fill_null_median(f1_data['SpeedI1'])
f1_data['IsPersonalBest'] = fill_null_mode(f1_data['IsPersonalBest'])
f1_data['Sector2SessionTime'] = fill_null_mode(f1_data['Sector2SessionTime'])
f1_data['Sector2Time'] = fill_null_mode(f1_data['Sector2Time'])
f1_data['Sector3SessionTime'] = fill_null_mode(f1_data['Sector3SessionTime'])
f1_data['Sector3Time'] = fill_null_mode(f1_data['Sector3Time'])
f1_data['LapTime'] = fill_null_mode(f1_data['LapTime'])
f1_data['Sector1Time'] = fill_null_mode(f1_data['Sector1Time'])
f1_data['Sector1SessionTime'] = fill_null_mode(f1_data['Sector1SessionTime'])

In [ ]:
cek_null(f1_data)

### Handling Duplicate Value

In [ ]:
print("Jumlah duplikasi data : " + str(f1_data.duplicated().sum()))

In [ ]:
f1_data = f1_data.drop_duplicates()

### Handling Outliers

In [ ]:
f1_data.boxplot(vert=False,figsize=(20,20))
plt.show()

In [ ]:
laptime_numerical = f1_data.select_dtypes(include=['float64', 'int64']) 
Q1 = laptime_numerical.quantile(0.25)
Q3 = laptime_numerical.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
print(cek_outlier(f1_data))

> Atribut `Presure` digunakan untuk memprediksi `TyreLife`, sehingga tidak kami drop .

### Encoding Categorical Attributes

In [ ]:
print("Menampilkan nilai unique yang terdapat pada setiap kolom kategorikal")
print('#'*70)
print()
for col in f1_data.select_dtypes(include=object).columns:
    print(col, f": {len(f1_data[col].unique())}", f1_data[col].unique())
    print("\n")

In [ ]:
f1_data['Compound'] = f1_data['Compound'].map({'INTERMEDIATE':1, 'MEDIUM':4, 'HARD':2, 'SOFT':3, 'WET':0})
f1_data['Pos_cat'] = f1_data['Pos_cat'].map({'Participant':0, 'Podium':2, 'Point':1})

In [ ]:
def zero_days_remover (duration):
    return duration.replace('0 days ', '')

In [ ]:
time_att = []
for col in f1_data.select_dtypes(include=object).columns:
    time_att.append(col)

In [ ]:
time_att

In [ ]:
for time in time_att:
    f1_data[time] = f1_data[time].apply(zero_days_remover)

Untuk memudahkan encoding waktu, kami menghilangkan 0 days karena seluruh data memiliki 0 days dan hal tersebut tidak membantu kami dalam melakukan encoding waktu

In [ ]:
import datetime
import time


In [ ]:
def time_converter (time_str):
    if '.' in time_str:
        time_str, milliseconds = time_str.split('.')
        milliseconds = int(milliseconds)
    else:
        milliseconds = 0

    x_time = datetime.datetime.strptime(time_str, '%H:%M:%S')

    total_seconds = datetime.timedelta(
        hours=x_time.hour,
        minutes=x_time.minute,
        seconds=x_time.second,
        microseconds=milliseconds / 1000  # Convert milliseconds to microseconds
    ).total_seconds()
    
    return total_seconds

In [ ]:
for time in time_att:
    f1_data[time] = f1_data[time].apply(time_converter)

In [ ]:
f1_data

In [ ]:
def convert_time_period(seconds):
    if 0 <= seconds < 6 * 3600:
        return 3   # From 00:00 to 05:59
    elif 6 * 3600 <= seconds < 12 * 3600:
        return 0  # From 06:00 to 11:59
    elif 12 * 3600 <= seconds < 17 * 3600:
        return 1  # From 12:00 to 16:59
    elif 17 * 3600 <= seconds < 21 * 3600:
        return 2  # From 17:00 to 20:59
    else:
        return 3  # From 21:00 to 23:59
f1_data['Time'] = f1_data['Time'].apply(convert_time_period)
f1_data['LapStartTime'] = f1_data['LapStartTime'].apply(convert_time_period)

f1_data

# **EKSPLORASI** 

Apakah penggunaan ‘Compound’ yang berbeda berpengaruh terhadap performa? 


Performa dari suatu mobil F1 dapat ditentukan melalui waktu yang ditempuh mobil tersebut selama satu lap, yaitu atribut LapTime. Atribut ini juga kami pilih sebagai representasi performa yang dianalisis pengaruhnya oleh Compound karena dalam balapan F1, tipe compound yang berbeda tidak mungkin dipakai dalam satu lap yang sama. Oleh karena itu pada eksplorasi ini kami hanya mengambil atribut Compound dan LapTime

In [ ]:
compount_influence = f1_data.copy()

In [ ]:
compount_influence = compount_influence[['Compound', 'LapTime']]
compount_influence

In [ ]:
compount_influence['Compound'] = compount_influence['Compound'].map({1: 'INTERMEDIATE', 4: 'MEDIUM', 2: 'HARD', 3: 'SOFT', 0: 'WET'})
compount_influence

In [ ]:
compounds = compount_influence['Compound'].unique()
for compound in compounds:
    plt.figure(figsize=(8, 6))
    subset = compount_influence[compount_influence['Compound'] == compound]['LapTime']
    sns.kdeplot(subset, fill=True)
    plt.title(f'KDE of Lap Times for {compound} Compound')
    plt.xlabel('Lap Time')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

In [ ]:
median_laptimes = compount_influence.groupby('Compound')['LapTime'].median()
median_laptimes = median_laptimes.sort_values()
median_laptimes.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Median Lap Times by Compound')
plt.xlabel('Compound Type')
plt.ylabel('Median Lap Time')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

> Grafik diatas adalah grafik yang menampilkan median LapTime dari setiap Compound dan diurutkan dari LapTime paling cepat hingga LapTime paling lambat

> Berdasarkan visualisasi diatas, dapat dianalisis bahwa setiap tipe Compound dapat mempengaruhi performa mobil F1 karena menghasilkan LapTime yang berbeda-beda. Performa terbaik didapatkan ketika mobil-mobil F1 menggunakan Compound bertipe SOFT. Lalu, diikuti dengan tipe Compound HARD, MEDIUM, & INTERMEDIATE untuk performa terbaik kedua, ketiga, dan keempat. Untuk performa terburuk, didapatkan ketika mobil F1 menggunakan tipe Compound WET.

Bagaimana ciri-ciri driver dengan kategori posisi ‘Pos_cat’ Podium dibandingkan dengan kategori posisi lainnya?


Analisis perbedaan ketika sirkuit hujan ‘Rainfall’ atau tidak


In [138]:
df_rain = f1_data.copy()
#plt.figure(figsize=(40,40))
#sns.heatmap(df_rain.corr(),annot=True)
#plt.show()
mean_attributes = ['LapTime', 'TyreLife', 'AirTemp', 'Humidity', 'TrackTemp','WindSpeed']
mode_attributes = ['Compound']

reverse_compound_mapping = {1: 'INTERMEDIATE', 4: 'MEDIUM', 2: 'HARD', 3: 'SOFT', 0: 'WET'}
df_rain['Compound'] = df_rain['Compound'].map(reverse_compound_mapping)

In [133]:
rainfall_data = df_rain[df_rain['Rainfall'] == True]
rainfall_mean = rainfall_data[mean_attributes].median()
rainfall_mode = rainfall_data[mode_attributes].mode().iloc[0]

rainfall_combined = pd.concat([rainfall_mean, rainfall_mode])
rainfall_combined


LapTime         100.00078
TyreLife              7.0
AirTemp              17.5
Humidity             66.0
TrackTemp            27.3
WindSpeed             1.7
Compound     INTERMEDIATE
dtype: object

In [134]:
no_rainfall_data = df_rain[df_rain['Rainfall'] == False]
no_rainfall_mean = no_rainfall_data[mean_attributes].median()
no_rainfall_mode = no_rainfall_data[mode_attributes].mode().iloc[0]

no_rainfall_combined = pd.concat([no_rainfall_mean, no_rainfall_mode])
no_rainfall_combined

LapTime      88.0006
TyreLife        12.0
AirTemp         25.6
Humidity        54.0
TrackTemp       34.3
WindSpeed        1.5
Compound        HARD
dtype: object

Di sini, kami akan melakukan perbandingan kondisi balapan ketika sirkuit 'Rainfall' dan tidak dengan asumsi bahwa data ini adalah kumpulan dari balapan-balapan satu musim di lokasi dan waktu yg berbeda. Dari data yang telah diolah, kami menemukan bahwa terdapat beberapa aspek yang berbeda dari sirkuit 'Rainfall' dan tidak.  LapTime dari dua keadaan sirkuit ini berbeda cukup signifikan, di mana dalam keadaan 'Rainfall' diperlukan lebih banyak waktu untuk melakukan 'LapTime' jika dibandingkan dengan track tidak 'Rainfall'. Dari segi 'AirTemp', sirkuit 'Rainfall' memiliki temperatur udara yang lebih rendah. Selain itu, sirkuit 'Rainfall' memiliki temperatur track yang lebih rendah, serta memiliki 'humidity' lebih tinggi jika dibanding track yang tidak 'Rainfall'. Ban Intermediate menjadi ban yang sering dipakai pada sirkuit 'Rainfall'. Di sirkuit yang tidak 'Rainfall', temperatur track lebih tinggi dibanding track 'Rainfall' dan memiliki 'WindDirection' lebih tinggi dibanding track 'Rainfall'. Adapun 'Compound' ban yang sering dipakai untuk sirkuit tidak 'Rainfall' adalah ban berjenis Hard. Kedua tipe sirkuit ini tidak berbeda terlalu jauh jika kita tinjau dari segi 'Pressure' atau tekanan udara.

Adakah rentang umur ban ‘TyreLife’ dengan performa terbaik dibandingkan rentang umur ban lainnya?


array(['LapTime', 'Stint', 'Sector2SessionTime', 'Sector3SessionTime',
       'Compound', 'TyreLife'], dtype=object)

# **REGRESI**

In [ ]:
f1_data

In [159]:
# https://medium.com/analytics-vidhya/evaluation-metrics-for-regression-models-c91c65d73af
def regression_metrics(prediction, y_test):
    MAE = mean_absolute_error(y_test, prediction)
    MSE = mean_squared_error(y_test, prediction)
    RMSE = np.sqrt(MSE)
    R_squared = r2_score(y_test, prediction)

    print('MAE: ' + str(MAE))
    print('MSE: ' + str(MSE))
    print('RMSE: ' + str(RMSE))
    print('R_squared: ' + str(R_squared))

In [161]:
# Split dan seleksi fitur Dataset Klasifikasi
X_regression_temp = f1_data.drop(columns=['TyreLife'], axis=1)
y_regression = f1_data['TyreLife']

mi = mutual_info_classif(X_regression_temp, y_regression)
mi = pd.Series(mi)
mi.index = X_regression_temp.columns
mi.sort_values(ascending=False)

KBest = math.ceil(0.2 * len(mi.index))

selector = SelectKBest(f_classif, k=KBest) 
X_regression = selector.fit_transform(X_regression_temp, y_regression)
input_features = selector.feature_names_in_
selector.get_feature_names_out(input_features=input_features)

array(['Time', 'Sector1SessionTime', 'Sector2SessionTime',
       'Sector3SessionTime', 'IsPersonalBest', 'LapStartTime'],
      dtype=object)

In [162]:
X_regress_train, X_regress_test, y_regress_train, y_regress_test = train_test_split(X_regression, y_regression, test_size=0.3, random_state=42)

In [163]:
#Standarisasi
scaler = MinMaxScaler()

X_regress_train = scaler.fit_transform(X_regress_train)
X_regress_test = scaler.transform(X_regress_test)

In [164]:
# Random Forest
rf_f1 = RandomForestRegressor()
rf_f1.fit(X_regress_train, y_regress_train)

# Memprediksi data testing
predicted = rf_f1.predict(X_regress_test)
regression_metrics(predicted, y_regress_test)

MAE: 6.137120378113992
MSE: 77.34947622957323
RMSE: 8.794855099975964
R_squared: 0.2608282015802822


In [165]:
alpha_f1 = [0.01, 0.1, 1, 10, 100]

In [167]:
ridge_model_f1 = []
for alpha in alpha_f1:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_regress_train, y_regress_train)
    ridge_model_f1.append(ridge)

In [173]:
from sklearn import metrics
metrics_list = []
counter = 1
for model in ridge_model_f1:
    y_pred = model.predict(X_regress_test)
    mse = metrics.mean_squared_error(y_regress_test, y_pred)
    mae = metrics.mean_absolute_error(y_regress_test, y_pred)
    rmse = math.sqrt(mse)
    r2 = metrics.r2_score(y_regress_test, y_pred)
    
    metrics_list.append((alpha_f1[counter-1], mse, mae, rmse, r2))
    
    print(f'Model Ridge regression ke {counter}')
    print("MSE:", mse)
    print("MAE:", mae)
    print("RMSE:", rmse)
    print("R-squared:", r2)
    print()
    counter += 1

# Find the best model based on RMSE (you can change the criteria if needed)
best_model_idx = min(range(len(metrics_list)), key=lambda i: metrics_list[i][3])  # Using RMSE for selection
best_model = ridge_model_f1[best_model_idx]

print(f'Model terbaik adalah model ke-{best_model_idx + 1} dengan alpha={metrics_list[best_model_idx][0]}')
print(f'Metrik Evaluasi - MSE: {metrics_list[best_model_idx][1]}, MAE: {metrics_list[best_model_idx][2]}, RMSE: {metrics_list[best_model_idx][3]}, R-squared: {metrics_list[best_model_idx][4]}')

Model Ridge regression ke 1
MSE: 81.7962100988712
MAE: 6.888393576178069
RMSE: 9.044125723300798
R-squared: 0.21833405124489613

Model Ridge regression ke 2
MSE: 81.78529252020888
MAE: 6.888197893738592
RMSE: 9.043522130243774
R-squared: 0.21843838247824887

Model Ridge regression ke 3
MSE: 81.94989425782168
MAE: 6.89039537093234
RMSE: 9.052618088587504
R-squared: 0.21686540527988818

Model Ridge regression ke 4
MSE: 82.33758539221103
MAE: 6.910074858994292
RMSE: 9.074006027781282
R-squared: 0.213160527535307

Model Ridge regression ke 5
MSE: 82.47068319951873
MAE: 6.920911352696065
RMSE: 9.081337082143726
R-squared: 0.21188861012372218

Model terbaik adalah model ke-2 dengan alpha=0.1
Metrik Evaluasi - MSE: 81.78529252020888, MAE: 6.888197893738592, RMSE: 9.043522130243774, R-squared: 0.21843838247824887


In [170]:
selected_ridge = ridge_model_f1[0]
selected_ridge

Ridge(alpha=0.01)

# **KLASIFIKASI**

Klasifikasi digunakan untuk membuat model dengan target `Pos_cat`

In [142]:
# Split Dataset Klasifikasi
X_classification_temp = f1_data.drop(columns=['Pos_cat'], axis=1)
y_classification = f1_data['Pos_cat']

mi = mutual_info_classif(X_classification_temp, y_classification)
mi = pd.Series(mi)
mi.index = X_classification_temp.columns
mi.sort_values(ascending=False)

KBest = math.ceil(0.2 * len(mi.index))

selector = SelectKBest(f_classif, k=KBest) 
X_classification = selector.fit_transform(X_classification_temp, y_classification)
input_features = selector.feature_names_in_
selector.get_feature_names_out(input_features=input_features)

array(['LapTime', 'Stint', 'Sector2SessionTime', 'Sector3SessionTime',
       'Compound', 'TyreLife'], dtype=object)

In [143]:
X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(X_classification, y_classification, test_size=0.3, random_state=42)

In [144]:
#Standarisasi
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_class_train)
X_test_scaled = scaler.transform(X_class_test)

In [151]:
# Decision Tree  
from sklearn.metrics import precision_score, \
    recall_score, classification_report, \
    accuracy_score, f1_score

def classification_metrics(prediction, y_test):
    print(f'Accuracy: {accuracy_score(y_test, prediction)}')
    print('F1 Macro Average:', f1_score(y_test, prediction, average='macro'))
    print('F1 Micro Average:', f1_score(y_test, prediction, average='micro'))
    print('Precision Macro Average:', precision_score(y_test, prediction, average='macro',zero_division=0))
    print('Precision Micro Average:', precision_score(y_test, prediction, average='micro',zero_division=0))
    print('Recall Macro Average:', recall_score(y_test, prediction, average='macro',zero_division=0))
    print('Recall Micro Average:', recall_score(y_test, prediction, average='micro',zero_division=0))
    print()

In [155]:
# Random Forest
rf_f1 = RandomForestClassifier()
rf_f1.fit(X_train_scaled, y_class_train)

# Memprediksi data testing
predicted = rf_f1.predict(X_test_scaled)
classification_metrics(predicted, y_class_test)


Accuracy: 0.7046301224055349
F1 Macro Average: 0.6649015164752367
F1 Micro Average: 0.7046301224055349
Precision Macro Average: 0.6790462028000986
Precision Micro Average: 0.7046301224055349
Recall Macro Average: 0.6558950652387289
Recall Micro Average: 0.7046301224055349



In [ ]:
# Decision Tree Hyperparameter

In [ ]:
# Random Forest Hyperparameter